In [1]:
from scipy import ndimage as ndi
from scipy.spatial import distance
import numpy as np


img = np.load('C:/Users/brian/Documents/thresholding.npy')
skeleton = np.load('C:/Users/brian/Documents/skeleton.npy')
skeleton = skeleton.astype(np.bool)
img = img.astype(np.bool)

distance_transform = ndi.distance_transform_edt(img)
distance_transform = distance_transform[skeleton]
distance_transform = distance_transform * 1.6
sort_index = np.argsort(distance_transform)
sort_index = np.flip(sort_index,axis=0)

i, j, k = np.mgrid[0:img.shape[0], 0:img.shape[1], 0:img.shape[2]]
i = i[skeleton]
j = j[skeleton]
k = k[skeleton]
l = 0
print(len(sort_index))
while l<(len(sort_index)-1):
    if(l%5000==0):
        print(l)
    mask = np.ones(len(sort_index),dtype=np.bool)
    radius = np.ceil(distance_transform[sort_index[l]])
    x = i[sort_index[l]]
    y = j[sort_index[l]]
    z = k[sort_index[l]]
    inner_loop_index = sort_index[l+1:]
    reverse_index1 = np.where((i[inner_loop_index]<x+radius) & (i[inner_loop_index]>x-radius))[0]
    inner_loop_index = inner_loop_index[reverse_index1]
    reverse_index2 = np.where((j[inner_loop_index]<y+radius) & (j[inner_loop_index]>y-radius))[0]
    inner_loop_index = inner_loop_index[reverse_index2]
    reverse_index3 = np.where((k[inner_loop_index]<z+radius) & (k[inner_loop_index]>z-radius))[0]
    inner_loop_index = inner_loop_index[reverse_index3]
    for inner in range(len(inner_loop_index)):
        if (distance_transform[sort_index[l]] - distance_transform[inner_loop_index[inner]])>distance.euclidean([x,y,z],[i[inner_loop_index[inner]],j[inner_loop_index[inner]],k[inner_loop_index[inner]]]):
            mask[reverse_index1[reverse_index2[reverse_index3[inner]]]+l+1] = False
    sort_index = sort_index[mask]
    l = l + 1

i = i[sort_index]
j = j[sort_index]
k = k[sort_index]

155265
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000


In [2]:
print(len(sort_index))


131531


In [3]:
from tifffile import imsave

scaled_distance_skeleton = np.zeros(img.shape)
for l in range(len(i)):
    scaled_distance_skeleton[i[l], j[l], k[l]] = 255
scaled_distance_skeleton = np.einsum('ijk->kij', scaled_distance_skeleton)
scaled_distance_skeleton = np.flip(scaled_distance_skeleton, axis=(1, 2))
imsave('C:/Users/brian/Documents/scaled_distance_skeleton.tif',scaled_distance_skeleton)